In [148]:
import csv
import collections
import statistics

In [200]:
with open('/global/scratch/users/pierrj/eccDNA/magnaporthe_pureculture/rawdata/illumina/G3_1A_tests/ecc_confidence') as coverage:
    coverage_reader = csv.reader(coverage, delimiter = '\t')
    coverage_list = [[] for i in range(56)]
    for row in coverage_reader:
        coverage_list[int(row[0])].append([int(row[1]), int(row[2]), str(row[3]), str(row[4]), str(row[5])])

In [ ]:
## name eccs (Chrom[num]ecc[num])
## maybe do this after actually

In [201]:
def uniq(lst):
    last = object()
    for item in lst:
        if item == last:
            continue
        yield item
        last = item

def sort_and_deduplicate(l):
    return list(uniq(sorted(l, key = lambda x: x[0])))

In [202]:
sortuniqcoverage_list_chrom0 = sort_and_deduplicate(coverage_list[20])

In [203]:
mydict = {}
for i in range(len(sortuniqcoverage_list_chrom0)):
    ecc = sortuniqcoverage_list_chrom0[i]
    lst = []
## THIS HAS TO HAS TO BE BOTH NOT ONE OR THE OTHER
    for k in range(len(sortuniqcoverage_list_chrom0)):
        coordinate1 = sortuniqcoverage_list_chrom0[k][0] - 50
        coordinate2 = sortuniqcoverage_list_chrom0[k][0] + 50
        if coordinate1 <= ecc[0] <= coordinate2:
            lst.append(sortuniqcoverage_list_chrom0[k])
    if tuple([ecc]) != tuple(lst):
        mydict[tuple(ecc)] = lst
val_map = collections.defaultdict(list)
for k,v in mydict.items():
    val_map[tuple(tuple(x) for x in v)].append(k)

In [204]:
def getcoord(coord_list):
    median = statistics.median(coord_list)
    distancetoend = abs(median - max(coord_list))
    distancetostart = abs(median - min(coord_list))
    greaterlist = [i for i in coord_list if i >= median]
    smallerlist = [i for i in coord_list if i <= median]
    if distancetoend > distancetostart:
        coord = min(smallerlist, key=lambda x:abs(x-median))
        coord_alt = 'N/A'
    elif distancetoend == distancetostart:
        coord = min(greaterlist, key=lambda x:abs(x-median))
        coord_alt = min(smallerlist, key=lambda x:abs(x-median))
    elif distancetoend < distancetostart:
        coord = min(greaterlist, key=lambda x:abs(x-median))
        coord_alt = 'N/A'
    else:
        raise ValueError("getcoord error")
    return coord, coord_alt

In [174]:
def test_coords(target_coord, other_coord, target_start_or_end):
        if target_start_or_end == 'start':
            known = 0
            unknown = 1
        if target_start_or_end == 'end':
            known = 1
            unknown = 0
        unknown_options = []
        for i in k:
            if i[known] == target_coord:
                unknown_options.append(i[unknown])
        unknown_coord = min(unknown_options, key=lambda x:abs(x-other_coord))
        return unknown_coord

def reconcile_coords(startcoord, endcoord):
        startcoord_forced = test_coords(startcoord, endcoord, 'start')
        endcoord_forced = test_coords(endcoord, startcoord, 'end')
        fit_startcoord_forced = abs(startcoord_forced - endcoord)
        fit_endcoord_forced = abs(endcoord_forced - startcoord)
        if fit_startcoord_forced <= fit_endcoord_forced:
            return startcoord, startcoord_forced
        else:
            return endcoord_forced, endcoord

In [175]:
for k in val_map.values():
    startlist = []
    endlist = []
    for i in k:
        startlist.append(i[0])
        endlist.append(i[1])
    #print(startlist)
    #print(endlist)
    #print(getcoord(startlist))
    #print(getcoord(endlist))
    start_coord, start_coord_alt = getcoord(startlist)
    end_coord, end_coord_alt = getcoord(endlist)
    middle_ecc = 0
    for i in k:
        if i[0] == start_coord and i[1] == end_coord:
            middle_ecc = i
        if i[0] == start_coord and i[1] == end_coord_alt:
            middle_ecc = i
        if i[0] == start_coord_alt and i[1] == end_coord:
            middle_ecc = i
        if i[0] == start_coord and i[1] == start_coord_alt:
            middle_ecc = i
    if middle_ecc == 0:
        #print("trying second pass")
        start_coord_estimate, end_coord_estimate = reconcile_coords(start_coord, end_coord)
        for i in k:
            if i[0] == start_coord_estimate and i[1] == end_coord_estimate:
                middle_ecc = i
    if middle_ecc == 0:
        raise ValueError("No ecc merged ecc called")
    else:
        print(middle_ecc)

(529, 909, 'conf', 'coverage_too_low', '36.4;39.02;36.88')
(10709, 11377, 'hconf', 'none', '5.65;33.65;13.88')
(13438, 14230, 'conf', 'coverage_too_low', '6.79;31.12;32.03')
(18781, 19701, 'conf', 'coverage_too_low', '41.42;40.0;30.63')
(22955, 23324, 'conf', 'coverage_too_low', '23.65;76.59;44.85')
(28826, 31817, 'conf', 'coverage_too_low', '73.6;59.67;32.14')
(30753, 33796, 'conf', 'coverage_too_low', '57.54;51.84;42.93')
(34948, 35283, 'conf', 'coverage_too_low', '9.75;30.13;38.38')
(35933, 38147, 'conf', 'coverage_too_low', '23.25;115.63;64.76')
(36030, 36406, 'conf', 'coverage_too_low', '47.18;94.41;88.1')
(37915, 39356, 'conf', 'coverage_too_low', '116.62;102.69;36.68')
(42094, 43744, 'hconf', 'none', '50.55;296.39;103.63')
(42266, 42988, 'conf', 'coverage_too_low', '86.68;363.21;263.89')
(44512, 45795, 'conf', 'coverage_too_low', '182.58;63.9;41.71')
(50694, 51611, 'conf', 'coverage_too_low', '53.34;120.2;70.7')
(50981, 51790, 'hconf', 'none', '56.44;143.52;53.94')
(51908, 52413

In [160]:
## so now we have a list of representative eccs
## now we need a dictionary with the key being the representative eccs

##something like
## loop through valmap.items() instead with key, val set
## then just val_map[middle_ecc] = val_map.pop(key)

for key, val in val_map.items():
    print(key)
    print(val)


((529, 936, 'conf', 'coverage_too_low', '35.99;38.72;37.25'), (529, 909, 'conf', 'coverage_too_low', '36.4;39.02;36.88'))
[(529, 936, 'conf', 'coverage_too_low', '35.99;38.72;37.25'), (529, 909, 'conf', 'coverage_too_low', '36.4;39.02;36.88')]
((10709, 11390, 'hconf', 'none', '5.62;33.61;13.68'), (10709, 11377, 'hconf', 'none', '5.65;33.65;13.88'))
[(10709, 11390, 'hconf', 'none', '5.62;33.61;13.68'), (10709, 11377, 'hconf', 'none', '5.65;33.65;13.88')]
((13438, 14236, 'conf', 'coverage_too_low', '6.82;31.67;31.16'), (13438, 14230, 'conf', 'coverage_too_low', '6.79;31.12;32.03'))
[(13438, 14236, 'conf', 'coverage_too_low', '6.82;31.67;31.16'), (13438, 14230, 'conf', 'coverage_too_low', '6.79;31.12;32.03')]
((18781, 19684, 'conf', 'coverage_too_low', '41.87;40.36;30.29'), (18781, 19701, 'conf', 'coverage_too_low', '41.42;40.0;30.63'))
[(18781, 19684, 'conf', 'coverage_too_low', '41.87;40.36;30.29'), (18781, 19701, 'conf', 'coverage_too_low', '41.42;40.0;30.63')]
((22955, 23324, 'conf', 

In [205]:
def test_coords(target_coord, other_coord, target_start_or_end):
        if target_start_or_end == 'start':
            known = 0
            unknown = 1
        if target_start_or_end == 'end':
            known = 1
            unknown = 0
        unknown_options = []
        for i in val:
            if i[known] == target_coord:
                unknown_options.append(i[unknown])
        unknown_coord = min(unknown_options, key=lambda x:abs(x-other_coord))
        return unknown_coord

def reconcile_coords(startcoord, endcoord):
        startcoord_forced = test_coords(startcoord, endcoord, 'start')
        endcoord_forced = test_coords(endcoord, startcoord, 'end')
        fit_startcoord_forced = abs(startcoord_forced - endcoord)
        fit_endcoord_forced = abs(endcoord_forced - startcoord)
        if fit_startcoord_forced <= fit_endcoord_forced:
            return startcoord, startcoord_forced
        else:
            return endcoord_forced, endcoord

In [206]:
newdict = {}
for key, val in val_map.items():
    startlist = []
    endlist = []
    for i in val:
        startlist.append(i[0])
        endlist.append(i[1])
    #print(startlist)
    #print(endlist)
    #print(getcoord(startlist))
    #print(getcoord(endlist))
    start_coord, start_coord_alt = getcoord(startlist)
    end_coord, end_coord_alt = getcoord(endlist)
    middle_ecc = 0
    for i in val:
        if i[0] == start_coord and i[1] == end_coord:
            middle_ecc = i
        if i[0] == start_coord and i[1] == end_coord_alt:
            middle_ecc = i
        if i[0] == start_coord_alt and i[1] == end_coord:
            middle_ecc = i
        if i[0] == start_coord and i[1] == start_coord_alt:
            middle_ecc = i
    if middle_ecc == 0:
        #print("trying second pass")
        start_coord_estimate, end_coord_estimate = reconcile_coords(start_coord, end_coord)
        for i in val:
            if i[0] == start_coord_estimate and i[1] == end_coord_estimate:
                middle_ecc = i
    if middle_ecc == 0:
        raise ValueError("No ecc merged ecc called")
    else:
        print(middle_ecc)
        print(key)
        newdict[middle_ecc] = val

(529, 909, 'conf', 'coverage_too_low', '36.4;39.02;36.88')
((529, 936, 'conf', 'coverage_too_low', '35.99;38.72;37.25'), (529, 909, 'conf', 'coverage_too_low', '36.4;39.02;36.88'))
(10709, 11377, 'hconf', 'none', '5.65;33.65;13.88')
((10709, 11390, 'hconf', 'none', '5.62;33.61;13.68'), (10709, 11377, 'hconf', 'none', '5.65;33.65;13.88'))
(13438, 14230, 'conf', 'coverage_too_low', '6.79;31.12;32.03')
((13438, 14236, 'conf', 'coverage_too_low', '6.82;31.67;31.16'), (13438, 14230, 'conf', 'coverage_too_low', '6.79;31.12;32.03'))
(18781, 19701, 'conf', 'coverage_too_low', '41.42;40.0;30.63')
((18781, 19684, 'conf', 'coverage_too_low', '41.87;40.36;30.29'), (18781, 19701, 'conf', 'coverage_too_low', '41.42;40.0;30.63'))
(22955, 23324, 'conf', 'coverage_too_low', '23.65;76.59;44.85')
((22955, 23324, 'conf', 'coverage_too_low', '23.65;76.59;44.85'), (22955, 23338, 'conf', 'coverage_too_low', '24.3;77.13;41.38'), (22955, 23312, 'conf', 'coverage_too_low', '23.08;76.01;48.03'), (22955, 23346, '

In [207]:
print(newdict)

{(529, 909, 'conf', 'coverage_too_low', '36.4;39.02;36.88'): [(529, 936, 'conf', 'coverage_too_low', '35.99;38.72;37.25'), (529, 909, 'conf', 'coverage_too_low', '36.4;39.02;36.88')], (10709, 11377, 'hconf', 'none', '5.65;33.65;13.88'): [(10709, 11390, 'hconf', 'none', '5.62;33.61;13.68'), (10709, 11377, 'hconf', 'none', '5.65;33.65;13.88')], (13438, 14230, 'conf', 'coverage_too_low', '6.79;31.12;32.03'): [(13438, 14236, 'conf', 'coverage_too_low', '6.82;31.67;31.16'), (13438, 14230, 'conf', 'coverage_too_low', '6.79;31.12;32.03')], (18781, 19701, 'conf', 'coverage_too_low', '41.42;40.0;30.63'): [(18781, 19684, 'conf', 'coverage_too_low', '41.87;40.36;30.29'), (18781, 19701, 'conf', 'coverage_too_low', '41.42;40.0;30.63')], (22955, 23324, 'conf', 'coverage_too_low', '23.65;76.59;44.85'): [(22955, 23324, 'conf', 'coverage_too_low', '23.65;76.59;44.85'), (22955, 23338, 'conf', 'coverage_too_low', '24.3;77.13;41.38'), (22955, 23312, 'conf', 'coverage_too_low', '23.08;76.01;48.03'), (22955

In [ ]:
### count how many eccs are within 100 bp of them on either side
### pick the ones with the highest counts somehow?
## something about the length?
## make a dictonary where the key is the ecc and the values are the lists that ecc is next to, including itself
## then delete keys with duplicate values
## then keep key that is in the middle